In [ ]:
# This is a project that makes asynchronous requests using asyncio and requests
# Inspired by http://skipperkongen.dk/2016/09/09/easy-parallel-http-requests-with-python-and-asyncio/

In [1]:
import asyncio
import timeit
import os
import threading
import requests

In [2]:
def synchronous_requests(urls):
    responses = [ requests.get(url) for url in urls ]
    return responses

In [3]:
def get(url):
    print("For URL: {0} PID: {1} TID: {2}\n".format( url, os.getpid(), threading.get_ident() ))
    return requests.get(url)

In [4]:
async def asynchronous_requests(urls, data):
    loop = asyncio.get_event_loop()
    futures = [ loop.run_in_executor(None, get, url) for url in urls ]
    for future in futures:
        asyncio.ensure_future(future)
    data = [ None for future in futures ]
    for iter in range(len(futures)):
        data[iter] = await futures[iter]
    return data

In [5]:
urls = [ 'http://xkcd.com', 'http://www.google.com', 'http://www.reddit.com', 'http://www.facebook.com' ]
start = timeit.default_timer()
responses = synchronous_requests(urls)
end = timeit.default_timer()
print("Time elapsed:",end-start)
for iter in range(len(responses)):
    print("URL: {0} \t Content-Type: {1}".format( urls[iter], responses[iter].headers['Content-Type'] ))

Time elapsed: 6.370854656001029
URL: http://xkcd.com 	 Content-Type: text/html; charset=UTF-8
URL: http://www.google.com 	 Content-Type: text/html; charset=ISO-8859-1
URL: http://www.reddit.com 	 Content-Type: text/html; charset=UTF-8
URL: http://www.facebook.com 	 Content-Type: text/html; charset=UTF-8


In [6]:
start = timeit.default_timer()
loop = asyncio.get_event_loop()
responses = []
responses = loop.run_until_complete(asynchronous_requests(urls, responses))
end = timeit.default_timer()
print("Time elapsed:",end-start)
for iter in range(len(responses)):
    print("URL: {0} \t Content-Type: {1}".format( urls[iter], responses[iter].headers['Content-Type'] ))

For URL: http://xkcd.com PID: 5234 TID: 139625240479488

For URL: http://www.google.com PID: 5234 TID: 139624823387904

For URL: http://www.reddit.com PID: 5234 TID: 139624814995200

For URL: http://www.facebook.com PID: 5234 TID: 139624806602496

Time elapsed: 1.9853229260043008
URL: http://xkcd.com 	 Content-Type: text/html; charset=UTF-8
URL: http://www.google.com 	 Content-Type: text/html; charset=ISO-8859-1
URL: http://www.reddit.com 	 Content-Type: text/html; charset=UTF-8
URL: http://www.facebook.com 	 Content-Type: text/html; charset=UTF-8
